In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, date
import time
import re
import requests
from bs4 import BeautifulSoup

In [9]:
#For csrapping whole pages 
num = [1]
for i in range(0,240,10):
    num.append(i)
soups = [] 
for page_num in num:
    url = requests.get(f'https://www.yelp.co.uk/search?find_desc=Pubs&find_loc=Berlin%2C+Germany&start={page_num}')
    print(url)
    soups.append( BeautifulSoup(url.content, 'html.parser') )
    time.sleep(12)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [10]:
pub_names2 =[]
ratings2 = []
number_of_rating2 =[]
price_level2 = []
neighbourhood2 = []
for soup in soups:
    htags =soup.find_all("h3",class_="css-kagwww")
    for tags in htags:
        pub_names2.append(tags.text.replace('\xa0','').split('.')[1])
    rates = soup.find_all('span', class_='reviewCount__09f24__tnBk4 css-chan6m')
    for rate in rates:
        number_of_rating2.append(rate.text)
    price_level = []
    princing = soup.select('.css-dzq7l1')
    for price in princing:
        price_output = price.text
        price_level2.append(re.findall('€+',price_output))
    neighbour = soup.select('.css-dzq7l1')
    for neighb in neighbour:
        neighb_output= neighb.text
        neighbourhood2.append(re.findall('€+(\w+)',neighb_output))
    ratings = soup.find_all('div',{'role':'img'})
    for rate in ratings:
        ratings2.append(rate['aria-label'])





In [11]:
print(len(pub_names2))
print(len(price_level2))
print(len(neighbourhood2))
print(len(ratings))
print(len(number_of_rating2))

250
250
250
8
228


In [13]:
##For cleaning the price columns 
final_data =pd.DataFrame({'Names':pub_names2, 'Prices':price_level2,'Neighbourhoods':neighbourhood2,'Number_of_reviews':number_of_rating2,'Ratings':ratings2})
final_data=final_data.mask(final_data.applymap(str).eq('[]'))
final_data=final_data.explode(['Prices'])
final_data=final_data.explode(['Neighbourhoods'])
final_data

ValueError: All arrays must be of the same length

In [8]:
final_data.to_csv('scrapping_pub.csv', index=False)